# Custom map simulations

In this tutorial we will build a simulation from scratch.

We start by defining a `Band` that will determine our array's sensitivity to different spectra. We then generate an array by specifying a field of view, which will be populated by evenly-spaced beams of the given band.

In [ ]:
import maria
from maria.instrument import Band

f090 = Band(center=90, # in GHz
            width=20,
            sensitivity=3e-5,
            gain_error=5e-2) # in K sqrt(s)

f150 = Band(center=150,
            width=30,
            sensitivity=5e-5,
            knee=1e0,
            gain_error=5e-2)

In [ ]:
array = {"field_of_view": 0.5, "primary_size": 20, "bands": [f090, f150]}

instrument = maria.get_instrument(array=array)

instrument.plot()

As something to observe, we can download a map and construct a `map`. We also define a plan to do a daisy scan centered on the center of the map.

In [ ]:
from maria.io import fetch

map_filename = fetch("maps/big_cluster.fits")

input_map = maria.map.read_fits(filename=map_filename, 
                                index=1,
                                width=1, 
                                center=(150, 10), 
                                units="Jy/pixel")

input_map.to(units="K_RJ").plot()

In [ ]:
plan = maria.Plan(scan_pattern="daisy",
                  scan_options={"radius": 0.5, "speed": 0.1}, # in degrees
                  duration=660, # in seconds
                  sample_rate=50, # in Hz
                  scan_center=(150, 10),
                  frame="ra_dec")

plan.plot()

In [ ]:
sim = maria.Simulation(instrument, 
                       plan=plan,
                       site="llano_de_chajnantor", 
                       map=input_map,
                       atmosphere="2d",
                      )

In [ ]:
tod = sim.run()

In [ ]:
from maria.mappers import BinMapper

mapper = BinMapper(center=(150, 10), 
                   frame="ra_dec", 
                   width=1e0, 
                   height=1e0,
                   resolution=5e-3,
                   tod_preprocessing={
                        "window": {"name": "tukey", "kwargs": {"alpha": 0.1}},
                        "despline": {"knot_spacing": 5},
                        "remove_modes": {"modes_to_remove": [0]},
                        "filter": {"f_lower": 0.08},
                    },
                    map_postprocessing={
                        "gaussian_filter": {"sigma": 1},
                        "median_filter": {"size": 1},
                    },
                  )

mapper.add_tods(tod)

output_map = mapper.run()

In [ ]:
output_map.plot()